# Freebase to PostgreSQL

This script is to move the data from files to postgreSQL to help analyze.

In [6]:
import sys
sys.path.insert(0, '../../')

In [7]:
from scripts.utils.connect import get_connection 

connection = get_connection()
cursor = connection.cursor()

## Index FB5M KG

In [5]:
fb_kg = '../../data/SimpleQuestions_v2/freebase-subsets/freebase-FB5M.txt'

In [6]:
cursor.execute("""
    CREATE TABLE fb_kg
        (object_mid varchar NOT NULL,
        relation varchar NOT NULL,
        subject_mid varchar NOT NULL,
        PRIMARY KEY(object_mid, relation, subject_mid));
    CREATE INDEX relation_index ON fb_kg (relation);
    CREATE INDEX subject_mid_index ON fb_kg (subject_mid);""")

In [7]:
from tqdm import tqdm_notebook

chunk_size = 20000


def insert_chunk(rows):
    insert_query = 'INSERT INTO fb_kg (object_mid, relation, subject_mid) VALUES %s ON CONFLICT DO NOTHING;'
    psycopg2.extras.execute_values(
        cursor, insert_query, rows, template=None, page_size=100
    )
    

rows = []
for line in tqdm_notebook(open(fb_kg, 'r'), total=12010500):
    # Build Chunks
    split = line.split('\t')
    assert len(split) == 3, 'Malformed row'
    subject = split[0].replace('www.freebase.com/m/', '').strip()
    property_ = split[1].replace('www.freebase.com/', '').strip()
    objects = [url.replace('www.freebase.com/m/', '').strip() for url in split[2].split()]
    rows.extend([tuple([str(object_), str(property_), str(subject)]) for object_ in objects])

    # Insert Chunk
    if len(rows) > chunk_size:
        insert_chunk(rows)
        rows = []
        
insert_chunk(rows)

HBox(children=(IntProgress(value=0, max=12010500), HTML(value='')))

In [8]:
connection.commit()

#### Some quick numbers:

- Total 12,010,500 FB5M tuples. Coraborated by https://research.fb.com/wp-content/uploads/2016/11/large-scale_simple_question_answering_with_memory_networks.pdf?
- Subject Unique MIDs ~ 3,929,894
- Subject MIDs ~ 46,424,988
- Object Unique MIDs ~ 1,972,702
- Object MIDs ~ 12,010,500. Coraborated by https://research.fb.com/wp-content/uploads/2016/11/large-scale_simple_question_answering_with_memory_networks.pdf?
- Total Unique MIDs ~ 3,988,105 ~ Coraborated by https://github.com/zihangdai/CFO
- Total MIDs ~ 58,435,488
- Total Relations ~ 7,523. Coraborated by https://research.fb.com/wp-content/uploads/2016/11/large-scale_simple_question_answering_with_memory_networks.pdf?
- Total Unique Facts ~ 17,872,174. Coraborated by https://github.com/zihangdai/CFO 


## Index FB5M Mid to Names

In [2]:
def get_all_mids():
    """ Get all MIDs in the FB5M KG """
    all_mids = set()
    cursor.execute("""SELECT object_mid FROM fb_kg""")
    [all_mids.add(mid) for (mid,) in cursor.fetchall()]
    cursor.execute("""SELECT subject_mid FROM fb_kg""")
    [all_mids.add(mid) for (mid,) in cursor.fetchall()]
    return all_mids

all_mids = get_all_mids()
print('Got %d mids' % len(all_mids))
print('Sample:', list(all_mids)[:5])

Got 3988105 mids
Sample: ['02x0zwh', '0bvmc8d', '01rpt3x', '0crvlj9', '0r4jpzg']


In [19]:
cursor.execute("""
    CREATE TABLE fb_name
        (mid varchar NOT NULL,
        alias varchar NOT NULL,
        PRIMARY KEY(mid, alias));""")

In [21]:
from tqdm import tqdm_notebook

chunk_size = 10000

def insert_chunk(rows):
    insert_query = 'INSERT INTO fb_name (mid, alias) VALUES %s ON CONFLICT DO NOTHING;'
    psycopg2.extras.execute_values(
        cursor, insert_query, rows, template=None
    )
    
def get_aliases(mid):
    sql = """
        SELECT subject
        FROM fb
        WHERE object = '<http://rdf.freebase.com/ns/m.%s>'
        AND subject LIKE '%%@en'
        AND relation IN ('<http://rdf.freebase.com/ns/type.object.name>',
                         '<http://rdf.freebase.com/ns/common.topic.alias>',
                         '<http://www.w3.org/2000/01/rdf-schema#label>',
                         '<http://rdf.freebase.com/ns/medicine.drug_formulation.brand_names>')
        """ % (mid,)
    cursor.execute(sql)
    aliases = [row[0].replace('@en', '').strip('"').lower() for row in cursor.fetchall()]
    # NOTE: Questions are all under 300 characters some aliases are not
    return set([a for a in aliases if len(a) < 300])

def get_replace_mid(mid):
    # MID may have been replaced if there are no aliases 
    # <http://rdf.freebase.com/ns/dataworld.gardening_hint.replaced_by>
    sql = """
        SELECT subject
        FROM fb
        WHERE object = '<http://rdf.freebase.com/ns/m.%s>'
        AND relation = '<http://rdf.freebase.com/ns/dataworld.gardening_hint.replaced_by>'
    """ % (mid,)
    cursor.execute(sql)
    mids = [row[0] for row in cursor.fetchall()]
    assert len(mids) <= 1
    if len(mids) == 1:
        return mids[0].replace('<http://rdf.freebase.com/ns/m.', '').rstrip('>')
    return None

rows = []
for mid in tqdm_notebook(all_mids):
    # Build Chunks
    # TODO: <http://rdf.freebase.com/ns/dataworld.gardening_hint.replaced_by>
    aliases = get_aliases(mid)
    if len(aliases) == 0:
        other_mid = get_replace_mid(mid)
        if other_mid is not None:
            aliases = get_aliases(other_mid)
    
    rows.extend([tuple([mid, alias]) for alias in aliases])

    # Insert Chunk
    if len(rows) > chunk_size:
        insert_chunk(rows)
        rows = []
        
insert_chunk(rows)

HBox(children=(IntProgress(value=0, max=3988105), HTML(value='')))

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [22]:
connection.commit()

In [23]:
cursor.execute('CREATE INDEX fb_name_alias_index ON fb_name (alias);')
cursor.execute('CREATE INDEX fb_name_mid_index ON fb_name (mid);')
connection.commit()

## Index All FB

The dump of Freebase can be found here: https://developers.google.com/freebase

Here are some of the ids in Freebase: https://stackoverflow.com/questions/24272840/freebase-g-vs-m-namespace-ids

### Sample

Here we investigate what `freebase-rdf-latest.gz` looks like. 

In [3]:
fb = '../../data/freebase-rdf-latest.gz'

In [ ]:
import random
import gzip
import io
from tqdm import tqdm_notebook

index = 0
gzip = subprocess.Popen(['gzip', '-cdfq', fb], stdout=subprocess.PIPE)
for line in tqdm_notebook(gzip.stdout, total=3129197591):
    # Build Chunks
    line = line.decode('UTF-8')
    split = line.split('\t')[:3]
    if random.random() < 0.0000001:
        print('| ' + str(index) +' | ' + str(' | '.join(split).encode('utf-8')) + ' |')
        index += 1

#### Results

| Index | Subject | Predicate | Object | 
|:--:|:--:|:--:|:--:|
| 0 | <http://rdf.freebase.com/ns/m.01nvvl> | <http://rdf.freebase.com/key/wikipedia.en> | "Russell_D\$002E_Feingold" |
| 1 | <http://rdf.freebase.com/ns/m.0pywvsb> | <http://rdf.freebase.com/ns/award.award_honor.award_winner> | <http://rdf.freebase.com/ns/m.09l6vp> |
| 2 | <http://rdf.freebase.com/ns/m.04b6sfs> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/location.geocode> |
| 3 | <http://rdf.freebase.com/ns/m.04vs9f5> | <http://rdf.freebase.com/ns/common.topic.topic_equivalent_webpage> | <http://madcat.library.wisc.edu/cgi-bin/Pwebrecon.cgi?BBID=4366597> |
| 4 | <http://rdf.freebase.com/ns/g.1254y8yd2> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Art"@no |
| 5 | <http://rdf.freebase.com/ns/g.12565x58d> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Kulutustavara"@fi |
| 6 | <http://rdf.freebase.com/ns/g.1yp39_vms> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Σύνθεση"@el |
| 7 | <http://rdf.freebase.com/ns/g.125bc6w3d> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Cinéma mondial"@fr |
| 8 | <http://rdf.freebase.com/ns/g.125ccrn20> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "اپیزود تلویزیون"@fa |
| 9 | <http://rdf.freebase.com/ns/g.1q3sgd9z9> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musikkspor"@no |
| 10 | <http://rdf.freebase.com/ns/location.mailing_address> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0dj08ds> |
| 11 | <http://rdf.freebase.com/ns/g.1259sz6j_> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Livro"@pt |
| 12 | <http://rdf.freebase.com/ns/m.0zn_fz> | <http://rdf.freebase.com/key/authority.musicbrainz.recording> | "277c231b-d64b-401d-8021-0ede0790bb3d" |
| 13 | <http://rdf.freebase.com/ns/g.125bs_m_9> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "ISBN"@en |
| 14 | <http://rdf.freebase.com/ns/m.0ls4ksg> | <http://rdf.freebase.com/ns/music.release_track.track_number> | "14" |
| 15 | <http://rdf.freebase.com/ns/g.125by8p4p> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "ISBN"@en-GB |
| 16 | <http://rdf.freebase.com/ns/g.1yg9666qc> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Faixa musical"@pt-PT |
| 17 | <http://rdf.freebase.com/ns/g.1256y31h0> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "منتج المستهلك"@ar |
| 18 | <http://rdf.freebase.com/ns/m.0cg5914> | <http://rdf.freebase.com/ns/film.film.other_crew> | <http://rdf.freebase.com/ns/m.0w_2m5j> |
| 19 | <http://rdf.freebase.com/ns/m.0psd1yj> | <http://rdf.freebase.com/ns/base.schemastaging.nutrition_information.per_quantity_description> | "lb"@en |
| 20 | <http://rdf.freebase.com/ns/base.type_ontology.non_agent> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0zff_8> |
| 21 | <http://rdf.freebase.com/ns/base.type_ontology.non_agent> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.01lbx9w> |
| 22 | <http://rdf.freebase.com/ns/g.1q5zdjbl5> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Miesto"@sk |
| 23 | <http://rdf.freebase.com/ns/g.125byfbjs> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Grāmatas izdevums"@lv |
| 24 | <http://rdf.freebase.com/ns/m.0t3r1jp> | <http://rdf.freebase.com/ns/type.object.permission> | <http://rdf.freebase.com/ns/m.019h> |
| 25 | <http://rdf.freebase.com/ns/g.1258s3tqh> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Personatge de ficció"@ca |
| 26 | <http://rdf.freebase.com/ns/base.type_ontology.physically_instantiable> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.026stf4> |
| 27 | <http://rdf.freebase.com/ns/g.12h30l4d9> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "شخصية في فيلم"@ar |
| 28 | <http://rdf.freebase.com/ns/m.0vx_hnd> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/music.track_contribution> |
| 29 | <http://rdf.freebase.com/ns/g.125c7hc6n> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Utwór muzyczny"@pl |
| 30 | <http://rdf.freebase.com/ns/m.0gj0vv2> | <http://rdf.freebase.com/ns/tv.tv_guest_role.episodes_appeared_in> | <http://rdf.freebase.com/ns/m.05dq8qz> |
| 31 | <http://rdf.freebase.com/ns/g.1255n5q3f> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Edição do livro"@pt |
| 32 | <http://rdf.freebase.com/ns/g.1z2srwd6g> | <http://rdf.freebase.com/ns/common.notable_for.notable_object> | <http://rdf.freebase.com/ns/m.0kpv11> |
| 33 | <http://rdf.freebase.com/ns/m.01d72k7> | <http://rdf.freebase.com/ns/common.topic.notable_types> | <http://rdf.freebase.com/ns/m.0kpv11> |
| 34 | <http://rdf.freebase.com/ns/m.042rf8m> | <http://rdf.freebase.com/ns/type.content.media_type> | <http://rdf.freebase.com/ns/m.08x8> |
| 35 | <http://rdf.freebase.com/ns/m.0jkym5r> | <http://rdf.freebase.com/ns/common.topic.notable_for> | <http://rdf.freebase.com/ns/g.125b3gx58> |
| 36 | <http://rdf.freebase.com/ns/m.0w7kb> | <http://rdf.freebase.com/ns/location.hud_foreclosure_area.total_residential_addresses> | <http://rdf.freebase.com/ns/m.07h5__s> |
| 37 | <http://rdf.freebase.com/ns/g.11b75r5bm_> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "اپیزود تلویزیون"@fa |
| 38 | <http://rdf.freebase.com/ns/m.02l3218> | <http://rdf.freebase.com/ns/common.topic.notable_types> | <http://rdf.freebase.com/ns/m.02kl684> |
| 39 | <http://rdf.freebase.com/ns/m.0zsl63b> | <http://rdf.freebase.com/ns/common.topic.notable_for> | <http://rdf.freebase.com/ns/g.1ypm_jcnq> |
| 40 | <http://rdf.freebase.com/ns/g.1254xc21s> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Kompakta disko"@eo |
| 41 | <http://rdf.freebase.com/ns/g.12blswqz4> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 42 | <http://rdf.freebase.com/ns/g.126stq8tj> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Canción"@es-419 |
| 43 | <http://rdf.freebase.com/ns/g.125bn8xq7> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 44 | <http://rdf.freebase.com/ns/g.125cr6j8n> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Телеэпизод"@ru |
| 45 | <http://rdf.freebase.com/ns/m.01brfqj> | <http://rdf.freebase.com/key/authority.musicbrainz> | "9485cdef-13d7-49c3-aaac-cf7d08037e58" |
| 46 | <http://rdf.freebase.com/ns/m.0bs6rbv> | <http://rdf.freebase.com/key/wikipedia.nl_title> | "Colus_glaber" |
| 47 | <http://rdf.freebase.com/ns/m.0fbdx_j> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.topic> |
| 48 | <http://rdf.freebase.com/ns/m.03tzjvw> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.image> |
| 49 | <http://rdf.freebase.com/ns/m.0_644mn> | <http://rdf.freebase.com/ns/music.composition.recordings> | <http://rdf.freebase.com/ns/m.01168dj> |
| 50 | <http://rdf.freebase.com/ns/m.0frz9lr> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.topic> |
| 51 | <http://rdf.freebase.com/ns/g.1yg93xshq> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musikkspor"@no |
| 52 | <http://rdf.freebase.com/ns/g.11b61jpptr> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Müzik Parçası"@tr |
| 53 | <http://rdf.freebase.com/ns/m.0fqzjj7> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/base.type_ontology.non_agent> |
| 54 | <http://rdf.freebase.com/ns/g.125920h_3> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "奥运会竞赛"@zh |
| 55 | <http://rdf.freebase.com/ns/m.05kvxmg> | <http://rdf.freebase.com/ns/type.content.uploaded_by> | <http://rdf.freebase.com/ns/m.0220tm0> |
| 56 | <http://rdf.freebase.com/ns/m.0f6t5> | <http://rdf.freebase.com/ns/type.object.key> | "/wikipedia/en/Symbolic_algebra" |
| 57 | <http://rdf.freebase.com/ns/m.05b3cq3> | <http://rdf.freebase.com/ns/type.object.key> | "/authority/imdb/name/nm4906598" |
| 58 | <http://rdf.freebase.com/ns/m.060zrcz> | <http://rdf.freebase.com/ns/common.webpage.uri> | <http://www.metrolyrics.com/in-the-sand-lyrics-anouk.html> |
| 59 | <http://rdf.freebase.com/ns/m.08286sx> | <http://rdf.freebase.com/ns/common.topic.alias> | "Book of new words."@en |
| 60 | <http://rdf.freebase.com/ns/g.1258h82bs> | <http://rdf.freebase.com/ns/common.notable_for.predicate> | "/type/object/type" |
| 61 | <http://rdf.freebase.com/ns/m.0lz9sn> | <http://rdf.freebase.com/ns/dataworld.gardening_hint.replaced_by> | <http://rdf.freebase.com/ns/m.0y91zx> |
| 62 | <http://rdf.freebase.com/ns/m.0nt796s> | <http://rdf.freebase.com/ns/music.release.track_list> | <http://rdf.freebase.com/ns/m.0nt79g5> |
| 63 | <http://rdf.freebase.com/ns/g.1258b7hq8> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "ISBN"@pt |
| 64 | <http://rdf.freebase.com/ns/g.12597wlz1> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "קטע מוסיקלי"@iw |
| 65 | <http://rdf.freebase.com/ns/m.0dv4m1_> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.non_agent> |
| 66 | <http://rdf.freebase.com/ns/g.11b60zrryd> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 67 | <http://rdf.freebase.com/ns/m.0fsz1x6> | <http://rdf.freebase.com/ns/music.recording.length> | "107.0" |
| 68 | <http://rdf.freebase.com/ns/g.126tpvn28> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musiknummer"@da |
| 69 | <http://rdf.freebase.com/ns/g.1yn5cgwfc> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Beste"@tr |
| 70 | <http://rdf.freebase.com/ns/m.02w2l2b> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/base.type_ontology.inanimate> |
| 71 | <http://rdf.freebase.com/ns/g.125g2l3ft> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "מוצר צריכה"@iw |
| 72 | <http://rdf.freebase.com/ns/m.07mr0> | <http://rdf.freebase.com/key/wikipedia.en_title> | "Tomaso_Albinoni" |
| 73 | <http://rdf.freebase.com/ns/m.01hdn5> | <http://rdf.freebase.com/ns/common.topic.topic_equivalent_webpage> | <http://fr.wikipedia.org/wiki/index.html?curid=419208> |
| 74 | <http://rdf.freebase.com/ns/g.1yy60jtfx> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musical Track"@fil |
| 75 | <http://rdf.freebase.com/ns/base.type_ontology.abstract> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0dqck5x> |
| 76 | <http://rdf.freebase.com/ns/base.type_ontology.abstract> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.048g7rn> |
| 77 | <http://rdf.freebase.com/ns/g.125dm7rxy> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "ISBN"@bg |
| 78 | <http://rdf.freebase.com/ns/m.06cvfhf> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/book.written_work> |
| 79 | <http://rdf.freebase.com/ns/m.0j74fzs> | <http://rdf.freebase.com/ns/type.content_import.uri> | <http://www.thetvdb.com/banners/episodes/71978/350533.jpg> |
| 80 | <http://rdf.freebase.com/ns/g.1yl5qw30v> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Person"@en-GB |
| 81 | <http://rdf.freebase.com/ns/m.0g_747> | <http://rdf.freebase.com/key/wikipedia.fr_id> | "1252732" |
| 82 | <http://rdf.freebase.com/ns/m.0rr3fpc> | <http://rdf.freebase.com/ns/common.topic.topic_equivalent_webpage> | <http://www.discogs.com/label/Welovejazz> |
| 83 | <http://rdf.freebase.com/ns/g.125661nth> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Trek Musik"@id |
| 84 | <http://rdf.freebase.com/ns/g.1254yrxyn> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Έκδοση βιβλίου"@el |
| 85 | <http://rdf.freebase.com/ns/g.1yg9x8fwy> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Runut Muzik"@ms |
| 86 | <http://rdf.freebase.com/ns/m.084fxz> | <http://rdf.freebase.com/key/wikipedia.zh-tw> | "\$6C1F\$4EFF" |
| 87 | <http://rdf.freebase.com/ns/m.0h68j_l> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/location.statistical_region> |
| 88 | <http://rdf.freebase.com/ns/measurement_unit.dated_percentage> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/g.1q63lg3_w> |
| 89 | <http://rdf.freebase.com/ns/g.126ssb89t> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musiikkikappale"@fi |
| 90 | <http://rdf.freebase.com/ns/g.11b75s2qws> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Skladba"@sk |
| 91 | <http://rdf.freebase.com/ns/g.125crtwf7> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musiknummer"@da |
| 92 | <http://rdf.freebase.com/ns/g.125689cy5> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "대중 교통 정류장"@ko |
| 93 | <http://rdf.freebase.com/ns/m.0c6trnz> | <http://rdf.freebase.com/key/authority.stanford.control> | "2119146" |
| 94 | <http://rdf.freebase.com/ns/sports.pro_athlete> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.05q76j5> |
| 95 | <http://rdf.freebase.com/ns/g.12565kslv> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Location"@en-GB |
| 96 | <http://rdf.freebase.com/ns/g.1257xhyb6> | <http://rdf.freebase.com/ns/common.notable_for.object> | <http://rdf.freebase.com/ns/m.03h314x> |
| 97 | <http://rdf.freebase.com/ns/g.125bpgn56> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Episodi de televisió"@ca |
| 98 | <http://rdf.freebase.com/ns/m.0145x63> | <http://rdf.freebase.com/ns/music.recording.artist> | <http://rdf.freebase.com/ns/m.01z7b3> |
| 99 | <http://rdf.freebase.com/ns/g.125b7l9xb> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Series literarias"@es-419 |
| 100 | <http://rdf.freebase.com/ns/m.0_jy466> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.inanimate> |
| 101 | <http://rdf.freebase.com/ns/g.126t1w9m1> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "संगीत ट्रैक"@hi |
| 102 | <http://rdf.freebase.com/ns/m.0dvcc9q> | <http://rdf.freebase.com/ns/type.object.name> | "Een borreltje hier een borreltje daar"@en |
| 103 | <http://rdf.freebase.com/ns/g.125cbjzlk> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "ISBN"@sr |
| 104 | <http://rdf.freebase.com/ns/type.content> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.03t4qhh> |
| 105 | <http://rdf.freebase.com/ns/g.12h3015zv> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Zeneszám"@hu |
| 106 | <http://rdf.freebase.com/ns/g.1254yr1d3> | <http://rdf.freebase.com/ns/common.notable_for.notable_object> | <http://rdf.freebase.com/ns/m.05zppz> |
| 107 | <http://rdf.freebase.com/ns/g.125d31zj6> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Veľkomesto/mesto/obec"@sk |
| 108 | <http://rdf.freebase.com/ns/g.12h2z21b0> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Лик из филма"@sr |
| 109 | <http://rdf.freebase.com/ns/g.12h34hjw2> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Runut Muzik"@ms |
| 110 | <http://rdf.freebase.com/ns/g.1255l13s2> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Futebol"@pt |
| 111 | <http://rdf.freebase.com/ns/g.125875ryw> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "المدينة/البلدة/القرية"@ar |
| 112 | <http://rdf.freebase.com/ns/g.1yfp0rvx3> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Müzik Parçası"@tr |
| 113 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.098hnb6> |
| 114 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0125jk5> |
| 115 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.06nz4l0> |
| 116 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0smxy50> |
| 117 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0sljh8l> |
| 118 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0ymh4j> |
| 119 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0shw9qk> |
| 120 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0gdwykp> |
| 121 | <http://rdf.freebase.com/ns/common.topic> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0zhk4g9> |
| 122 | <http://rdf.freebase.com/ns/g.1259fp1jw> | <http://rdf.freebase.com/ns/common.notable_for.predicate> | "/type/object/type" |
| 123 | <http://rdf.freebase.com/ns/g.1yn5brzd3> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "ساخت"@fa |
| 124 | <http://rdf.freebase.com/ns/business.stock_ticker_symbol> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0ckr_wk> |
| 125 | <http://rdf.freebase.com/ns/m.010sn7> | <http://rdf.freebase.com/ns/common.topic.topic_equivalent_webpage> | <http://en.wikipedia.org/wiki/index.html?curid=138115> |
| 126 | <http://rdf.freebase.com/ns/m.0g1rmrm> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.topic> |
| 127 | <http://rdf.freebase.com/ns/g.1259h0wr9> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Cod poştal"@ro |
| 128 | <http://rdf.freebase.com/ns/g.1yl5qqbzz> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Filmreeks"@nl |
| 129 | <http://rdf.freebase.com/ns/m.0sh3db> | <http://rdf.freebase.com/ns/type.object.key> | "/authority/musicbrainz/recording/7101d475-70da-4a7f-a121-928a7d96a7f3" |
| 130 | <http://rdf.freebase.com/ns/g.12h2_3x9b> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Faixa musical"@pt-PT |
| 131 | <http://rdf.freebase.com/ns/m.0zm42b7> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.non_agent> |
| 132 | <http://rdf.freebase.com/ns/g.11b75mjdpl> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Hudební vydavatelství"@cs |
| 133 | <http://rdf.freebase.com/ns/g.12550180h> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "مسار موسيقي"@ar |
| 134 | <http://rdf.freebase.com/ns/g.1q3sdldnd> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "แทร็คเพลง"@th |
| 135 | <http://rdf.freebase.com/ns/m.013cpzsx> | <http://rdf.freebase.com/ns/music.recording.artist> | <http://rdf.freebase.com/ns/m.0bmf8s0> |
| 136 | <http://rdf.freebase.com/ns/m.0fpg6xc> | <http://rdf.freebase.com/ns/type.object.name> | "斯泰因赫尔角"@zh |
| 137 | <http://rdf.freebase.com/ns/m.0g0rbrj> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.topic> |
| 138 | <http://rdf.freebase.com/ns/g.126shrdl3> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Müzik Parçası"@tr |
| 139 | <http://rdf.freebase.com/ns/m.0dn0fq4> | <http://rdf.freebase.com/ns/type.object.name> | "Feelee"@ig |
| 140 | <http://rdf.freebase.com/ns/m.06gx701> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/book.written_work> |
| 141 | <http://rdf.freebase.com/ns/m.0mxmxt6> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/film.film_regional_release_date> |
| 142 | <http://rdf.freebase.com/ns/g.1255c5m12> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Könyv kiadása"@hu |
| 143 | <http://rdf.freebase.com/ns/m.088rshy> | <http://rdf.freebase.com/ns/common.topic.topic_equivalent_webpage> | <http://books.google.com/books?id=Y99lAAAAMAAJ> |
| 144 | <http://rdf.freebase.com/ns/g.1yl5mg55k> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Tv-episode"@da |
| 145 | <http://rdf.freebase.com/ns/g.1yl5pzw97> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Klasifikácia organizmov"@sk |
| 146 | <http://rdf.freebase.com/ns/g.1259jd0yt> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "ISBN"@ca |
| 147 | <http://rdf.freebase.com/ns/g.125h7b9b9> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "International Standard Book Number: Số sách tiêu chuẩn quốc tế"@vi |
| 148 | <http://rdf.freebase.com/ns/m.0ybc_bx> | <http://www.w3.org/2000/01/rdf-schema#label> | "Contradictions amoureuses"@en |
| 149 | <http://rdf.freebase.com/ns/g.126sy10zv> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "音楽トラック"@ja |
| 150 | <http://rdf.freebase.com/ns/m.03d1v2m> | <http://rdf.freebase.com/ns/type.object.key> | "/wikipedia/en/Franz_Xavier_of_Saxony" |
| 151 | <http://rdf.freebase.com/ns/m.0dr2vpf> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.topic> |
| 152 | <http://rdf.freebase.com/ns/m.0l6f2j> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.topic> |
| 153 | <http://rdf.freebase.com/ns/m.0n2h2k0> | <http://www.w3.org/2000/01/rdf-schema#label> | "Death in Venice, Act II Scene 16 (The Last Visit to Venice): Hurrah for the Piazza"@en |
| 154 | <http://rdf.freebase.com/ns/g.1257sw0hr> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Музикален запис"@bg |
| 155 | <http://rdf.freebase.com/ns/m.0bb1nnx> | <http://rdf.freebase.com/ns/common.topic.notable_for> | <http://rdf.freebase.com/ns/g.125cpnqr_> |
| 156 | <http://rdf.freebase.com/ns/m.0k1kbnf> | <http://rdf.freebase.com/ns/common.topic.notable_types> | <http://rdf.freebase.com/ns/m.01xrzlb> |
| 157 | <http://rdf.freebase.com/ns/g.11b5mb6yn8> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.abstract> |
| 158 | <http://rdf.freebase.com/ns/g.125bd9kl6> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 159 | <http://rdf.freebase.com/ns/g.125h51tqw> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musical Track"@lt |
| 160 | <http://rdf.freebase.com/ns/g.126smcgcc> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 161 | <http://rdf.freebase.com/ns/g.12552m5d5> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musical Recording"@en |
| 162 | <http://rdf.freebase.com/ns/m.011kgxgc> | <http://rdf.freebase.com/ns/music.track_contribution.contributor> | <http://rdf.freebase.com/ns/m.0nhwwpz> |
| 163 | <http://rdf.freebase.com/ns/g.1255syb0m> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Rahvusvaheline raamatu standardnumber (ISBN)"@et |
| 164 | <http://rdf.freebase.com/ns/g.1yg97qgzq> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "संगीत ट्रैक"@hi |
| 165 | <http://rdf.freebase.com/ns/m.0bf8v2n> | <http://rdf.freebase.com/ns/kg.object_profile.prominent_type> | <http://rdf.freebase.com/ns/music.album> |
| 166 | <http://rdf.freebase.com/ns/m.012gy3gm> | <http://rdf.freebase.com/ns/common.topic.notable_types> | <http://rdf.freebase.com/ns/m.0kpv11> |
| 167 | <http://rdf.freebase.com/ns/m.010jptlj> | <http://rdf.freebase.com/ns/music.artist.track> | <http://rdf.freebase.com/ns/m.010jqqvh> |
| 168 | <http://rdf.freebase.com/ns/m.0by1z1z> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.inanimate> |
| 169 | <http://rdf.freebase.com/ns/g.126sh22nv> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Utwór muzyczny"@pl |
| 170 | <http://rdf.freebase.com/ns/m.03969wj> | <http://rdf.freebase.com/ns/music.release.track_list> | <http://rdf.freebase.com/ns/m.0lqx1fl> |
| 171 | <http://rdf.freebase.com/ns/m.0f2wv5t> | <http://rdf.freebase.com/key/authority.musicbrainz.recording> | "4da95b2c-f8db-4e84-8995-fce86cc8b24f" |
| 172 | <http://rdf.freebase.com/ns/g.11b7p06jqc> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Hudební stopa"@cs |
| 173 | <http://rdf.freebase.com/ns/m.0v0ndpr> | <http://rdf.freebase.com/ns/type.object.name> | "Freebase Review Queue"@en |
| 174 | <http://rdf.freebase.com/ns/g.1258jb0tl> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 175 | <http://rdf.freebase.com/ns/g.125bw9cw6> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Canción"@es-419 |
| 176 | <http://rdf.freebase.com/ns/base.type_ontology.inanimate> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.072pm_0> |
| 177 | <http://rdf.freebase.com/ns/g.1254y46ft> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Lanzamiento musical"@es-419 |
| 178 | <http://rdf.freebase.com/ns/g.1257wxhx1> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "رقم دولي معياري للكتاب (ISBN)"@ar |
| 179 | <http://rdf.freebase.com/ns/m.0g_y4mc> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.image> |
| 180 | <http://rdf.freebase.com/ns/m.0ksx15> | <http://rdf.freebase.com/ns/type.object.key> | "/wikipedia/fa_id/2608213" |
| 181 | <http://rdf.freebase.com/ns/g.11b8c3437b> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/music.release_track> |
| 182 | <http://rdf.freebase.com/ns/g.125dm__gr> | <http://rdf.freebase.com/ns/common.notable_for.object> | <http://rdf.freebase.com/ns/m.04lw> |
| 183 | <http://rdf.freebase.com/ns/g.1yg9c23vd> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Utwór muzyczny"@pl |
| 184 | <http://rdf.freebase.com/ns/m.0lk6_xw> | <http://rdf.freebase.com/ns/music.release_track.recording> | <http://rdf.freebase.com/ns/m.0180v27> |
| 185 | <http://rdf.freebase.com/ns/g.125gdn3_f> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 186 | <http://rdf.freebase.com/ns/m.05ddsgr> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.abstract> |
| 187 | <http://rdf.freebase.com/ns/m.0nyj_fm> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.topic> |
| 188 | <http://rdf.freebase.com/ns/m.0zkj5f4> | <http://rdf.freebase.com/ns/type.object.key> | "/authority/musicbrainz/work/f463c328-17ed-3746-86ef-7e98e8b644da" |
| 189 | <http://rdf.freebase.com/ns/g.125773_cr> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Hudobný album"@sk |
| 190 | <http://rdf.freebase.com/ns/m.05dtp78> | <http://rdf.freebase.com/ns/tv.tv_series_episode.episode_number> | "19" |
| 191 | <http://rdf.freebase.com/ns/g.125cmhst4> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "آلبوم موزیکال"@fa |
| 192 | <http://rdf.freebase.com/ns/m.0120p3td> | <http://rdf.freebase.com/ns/music.album.releases> | <http://rdf.freebase.com/ns/m.0120l7jy> |
| 193 | <http://rdf.freebase.com/ns/m.06q6mtt> | <http://rdf.freebase.com/ns/type.object.key> | "/en/linguistic_change_in_french" |
| 194 | <http://rdf.freebase.com/ns/g.1yg9mgw1d> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Canción"@es-419 |
| 195 | <http://rdf.freebase.com/ns/g.126sfkkyf> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Autor"@es |
| 196 | <http://rdf.freebase.com/ns/m.0mtjt9> | <http://rdf.freebase.com/ns/music.recording.artist> | <http://rdf.freebase.com/ns/m.039dp_> |
| 197 | <http://rdf.freebase.com/ns/m.04gbg1n> | <http://rdf.freebase.com/ns/type.object.id> | "/user/cesca" |
| 198 | <http://rdf.freebase.com/ns/m.0124r0xd> | <http://rdf.freebase.com/ns/music.release.album> | <http://rdf.freebase.com/ns/m.0124dg24> |
| 199 | <http://rdf.freebase.com/ns/m.0x2qllt> | <http://www.w3.org/2000/01/rdf-schema#label> | "Anna Theis"@en |
| 200 | <http://rdf.freebase.com/ns/m.01clspg> | <http://rdf.freebase.com/ns/music.recording.releases> | <http://rdf.freebase.com/ns/m.032klf_> |
| 201 | <http://rdf.freebase.com/ns/g.126t4q0jv> | <http://rdf.freebase.com/ns/common.notable_for.object> | <http://rdf.freebase.com/ns/m.0kpv11> |
| 202 | <http://rdf.freebase.com/ns/m.0wzpwz_> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/type.domain> |
| 203 | <http://rdf.freebase.com/ns/g.125cp6clq> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Skaņdarbs"@lv |
| 204 | <http://rdf.freebase.com/ns/m.0h66cfq> | <http://rdf.freebase.com/ns/common.topic.description> | "Kesertaş is a village in the District of Gerger, Adıyaman Province, Turkey."@en |
| 205 | <http://rdf.freebase.com/ns/m.0by39q> | <http://rdf.freebase.com/ns/common.topic.alias> | "Cerberus Capital Management LP"@ro |
| 206 | <http://rdf.freebase.com/ns/g.125g1z1v6> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 207 | <http://rdf.freebase.com/ns/m.04_0d8p> | <http://rdf.freebase.com/ns/type.object.key> | "/wikipedia/fr/SP-171" |
| 208 | <http://rdf.freebase.com/ns/m.0nm2z3r> | <http://rdf.freebase.com/key/authority.musicbrainz.recording> | "6902fe0c-b11a-4216-9d28-86e27988351d" |
| 209 | <http://rdf.freebase.com/ns/g.12578zxrm> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Hoger onderwijs"@nl |
| 210 | <http://rdf.freebase.com/ns/m.0gt4g_4> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.non_agent> |
| 211 | <http://rdf.freebase.com/ns/m.03f0wbn> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/music.musical_group> |
| 212 | <http://rdf.freebase.com/ns/m.0dgqgf1> | <http://rdf.freebase.com/ns/common.document.source_uri> | <http://wp/en/28975044> |
| 213 | <http://rdf.freebase.com/ns/m.0ll5ky8> | <http://rdf.freebase.com/ns/music.release_track.recording> | <http://rdf.freebase.com/ns/m.0n277y> |
| 214 | <http://rdf.freebase.com/ns/m.08sxj7> | <http://rdf.freebase.com/key/en> | "bugenasaura" |
| 215 | <http://rdf.freebase.com/ns/m.0ss7ds1> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/music.release_track> |
| 216 | <http://rdf.freebase.com/ns/g.1257tm1b6> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "哲学家"@zh |
| 217 | <http://rdf.freebase.com/ns/m.0fv2tst> | <http://www.w3.org/2000/01/rdf-schema#label> | "Farewell to the Roxy"@en |
| 218 | <http://rdf.freebase.com/ns/m.0shkl> | <http://rdf.freebase.com/ns/location.hud_county_place.place> | <http://rdf.freebase.com/ns/m.0shkl> |
| 219 | <http://rdf.freebase.com/ns/m.0nzj9dt> | <http://rdf.freebase.com/ns/film.film_crew_gig.crewmember> | <http://rdf.freebase.com/ns/m.0ld4zft> |
| 220 | <http://rdf.freebase.com/ns/m.0n49fql> | <http://rdf.freebase.com/ns/type.object.name> | "Blood bike"@en |
| 221 | <http://rdf.freebase.com/ns/music.release_track> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0skw7s9> |
| 222 | <http://rdf.freebase.com/ns/music.release_track> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0mtlvdv> |
| 223 | <http://rdf.freebase.com/ns/g.125bmmn2f> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "การเผยแพร่ดนตรี"@th |
| 224 | <http://rdf.freebase.com/ns/g.1259rjwjv> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Софтвер"@sr |
| 225 | <http://rdf.freebase.com/ns/g.1yl5zcdd4> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Gène"@fr |
| 226 | <http://rdf.freebase.com/ns/m.04116_f> | <http://rdf.freebase.com/key/wikipedia.ja_id> | "218674" |
| 227 | <http://rdf.freebase.com/ns/m.06445jh> | <http://rdf.freebase.com/ns/type.permission.controls> | <http://rdf.freebase.com/ns/m.06445kr> |
| 228 | <http://rdf.freebase.com/ns/m.07q2n9> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/people.person> |
| 229 | <http://rdf.freebase.com/ns/m.0l63g4x> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.topic> |
| 230 | <http://rdf.freebase.com/ns/common.notable_for> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/g.125dz6_cj> |
| 231 | <http://rdf.freebase.com/ns/common.notable_for> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/g.125c1jn01> |
| 232 | <http://rdf.freebase.com/ns/common.notable_for> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/g.1259zdrw7> |
| 233 | <http://rdf.freebase.com/ns/common.notable_for> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/g.126t24xms> |
| 234 | <http://rdf.freebase.com/ns/book.book_edition> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0bht16b> |
| 235 | <http://rdf.freebase.com/ns/g.1255gcyn6> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Watak Fiksyen"@ms |
| 236 | <http://rdf.freebase.com/ns/g.125cdv6wh> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Химическое соединение"@ru |
| 237 | <http://rdf.freebase.com/ns/g.12h32y5dp> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Elokuvan hahmo"@fi |
| 238 | <http://rdf.freebase.com/ns/g.125773mvn> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.notable_for> |
| 239 | <http://rdf.freebase.com/ns/m.0dnxbtr> | <http://rdf.freebase.com/key/authority.musicbrainz> | "36ce894e-33f2-4fdf-88e3-f655f4eb5a53" |
| 240 | <http://rdf.freebase.com/ns/m.0v34ws1> | <http://rdf.freebase.com/ns/type.object.name> | "Scum (live)"@en |
| 241 | <http://rdf.freebase.com/ns/m.0ydmglk> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/common.topic> |
| 242 | <http://rdf.freebase.com/ns/g.11b5m0327y> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "संगीत एल्बम"@hi |
| 243 | <http://rdf.freebase.com/ns/g.1q63q4ktz> | <http://rdf.freebase.com/ns/measurement_unit.dated_percentage.date> | "1999-08"^^<http://www.w3.org/2001/XMLSchema#gYearMonth> |
| 244 | <http://rdf.freebase.com/ns/m.07mchtr> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.topic> |
| 245 | <http://rdf.freebase.com/ns/m.0l32hmz> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/common.document> |
| 246 | <http://rdf.freebase.com/ns/g.1yg9grn3y> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "音楽トラック"@ja |
| 247 | <http://rdf.freebase.com/ns/m.01h0fq> | <http://rdf.freebase.com/ns/type.object.key> | "/wikipedia/pt_title/Capitel_do_Le\$00E3o_de_Asoca" |
| 248 | <http://rdf.freebase.com/ns/m.048hj9q> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/base.type_ontology.non_agent> |
| 249 | <http://rdf.freebase.com/ns/m.0pzlt0r> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/base.schemastaging.nutrition_information> |
| 250 | <http://rdf.freebase.com/ns/m.0y527jr> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.abstract> |
| 251 | <http://rdf.freebase.com/ns/m.0x98f22> | <http://rdf.freebase.com/ns/music.recording.length> | "199.0" |
| 252 | <http://rdf.freebase.com/ns/g.1s057nq77> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Hudební stopa"@cs |
| 253 | <http://rdf.freebase.com/ns/g.1yg4dx116> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Персонаж фільму"@uk |
| 254 | <http://rdf.freebase.com/ns/m.0g87b5p> | <http://rdf.freebase.com/ns/common.topic.topic_equivalent_webpage> | <http://musicbrainz.org/recording/a4546240-27fc-448f-a20c-0aae24fd8ced> |
| 255 | <http://rdf.freebase.com/ns/m.05bgdgn> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/type.content> |
| 256 | <http://rdf.freebase.com/ns/m.076562_> | <http://www.w3.org/2000/01/rdf-schema#label> | "Fairview Middle School"@en |
| 257 | <http://rdf.freebase.com/ns/m.08mr3kn> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/type.namespace> |
| 258 | <http://rdf.freebase.com/ns/g.1256g43gd> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Κωδικός ISBN"@el |
| 259 | <http://rdf.freebase.com/ns/m.02rntpp> | <http://rdf.freebase.com/ns/common.topic.topic_equivalent_webpage> | <http://fr.wikipedia.org/wiki/index.html?curid=3805977> |
| 260 | <http://rdf.freebase.com/ns/m.0c1s15> | <http://rdf.freebase.com/key/wikipedia.pt> | "Esquadr\$00E3o_Atari" |
| 261 | <http://rdf.freebase.com/ns/g.126sk423h> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Morceau"@fr |
| 262 | <http://rdf.freebase.com/ns/m.0mqrbwh> | <http://www.w3.org/2000/01/rdf-schema#label> | "ヒの字"@ja |
| 263 | <http://rdf.freebase.com/ns/g.125gpgwwh> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Відеогра"@uk |
| 264 | <http://rdf.freebase.com/ns/g.12yxhxfxb> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Hudobník"@sk |
| 265 | <http://rdf.freebase.com/ns/m.089_b_> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/business.employer> |
| 266 | <http://rdf.freebase.com/ns/g.125cr6h61> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "棟篤笑"@zh-TW |
| 267 | <http://rdf.freebase.com/ns/m.0143fxb> | <http://rdf.freebase.com/ns/kg.object_profile.prominent_type> | <http://rdf.freebase.com/ns/music.recording> |
| 268 | <http://rdf.freebase.com/ns/m.0k3p8hr> | <http://rdf.freebase.com/ns/common.topic.article> | <http://rdf.freebase.com/ns/m.0k3pbv0> |
| 269 | <http://rdf.freebase.com/ns/m.0nvwm> | <http://rdf.freebase.com/key/wikipedia.pt> | "Condado_de_Carroll_\$0028Illinois\$0029" |
| 270 | <http://rdf.freebase.com/ns/m.0lgb1n3> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/music.release_track> |
| 271 | <http://rdf.freebase.com/ns/music.album> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0fstzhf> |
| 272 | <http://rdf.freebase.com/ns/location.hud_foreclosure_area> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.071613n> |
| 273 | <http://rdf.freebase.com/ns/m.0115__xp> | <http://rdf.freebase.com/ns/type.object.type> | <http://rdf.freebase.com/ns/film.performance> |
| 274 | <http://rdf.freebase.com/ns/m.0ftnt1p> | <http://rdf.freebase.com/key/authority.discogs.release> | "2178887" |
| 275 | <http://rdf.freebase.com/ns/g.12570xj6c> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Episode TV"@id |
| 276 | <http://rdf.freebase.com/ns/m.010366l_> | <http://rdf.freebase.com/ns/award.award_nomination.award_nominee> | <http://rdf.freebase.com/ns/m.05h0v3> |
| 277 | <http://rdf.freebase.com/ns/g.11b75r82lr> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "뮤지컬 앨범"@ko |
| 278 | <http://rdf.freebase.com/ns/g.1256r4321> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Телевизионен епизод"@bg |
| 279 | <http://rdf.freebase.com/ns/m.0snws5c> | <http://rdf.freebase.com/ns/music.release_track.track_number> | "3" |
| 280 | <http://rdf.freebase.com/ns/m.05s4nq> | <http://rdf.freebase.com/ns/type.object.key> | "/wikipedia/pl_id/965325" |
| 281 | <http://rdf.freebase.com/ns/g.120xcww2> | <http://www.w3.org/2000/01/rdf-schema#label> | "Я бреду по бездорожью..."@ru |
| 282 | <http://rdf.freebase.com/ns/g.1256c6t4x> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Musiktrack"@de |
| 283 | <http://rdf.freebase.com/ns/g.126sphprl> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Bản nhạc"@vi |
| 284 | <http://rdf.freebase.com/ns/g.125b6pyln> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "رقم دولي معياري للكتاب (ISBN)"@ar |
| 285 | <http://rdf.freebase.com/ns/m.0c6kd4s> | <http://www.w3.org/2000/01/rdf-schema#label> | "James I."@en |
| 286 | <http://rdf.freebase.com/ns/g.1yg98z78l> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Bản nhạc"@vi |
| 287 | <http://rdf.freebase.com/ns/g.11b75qs0r3> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Tao"@fil |
| 288 | <http://rdf.freebase.com/ns/g.125bwrv06> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Personnage de fiction"@fr |
| 289 | <http://rdf.freebase.com/ns/m.0y6k6z3> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/film.performance> |
| 290 | <http://rdf.freebase.com/ns/g.125d3_rwy> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "TV Program"@en |
| 291 | <http://rdf.freebase.com/ns/m.06fqw97> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/base.type_ontology.abstract> |
| 292 | <http://rdf.freebase.com/ns/m.011wc4r8> | <http://rdf.freebase.com/ns/music.release.format> | <http://rdf.freebase.com/ns/m.0f255> |
| 293 | <http://rdf.freebase.com/ns/m.012bk119> | <http://rdf.freebase.com/ns/type.object.key> | "/dataworld/freeq/job_9a25f287-1d42-47ef-9ef3-91d73a8478ac_var_5b96f29dde8bab7e" |
| 294 | <http://rdf.freebase.com/ns/m.0mgswfn> | <http://rdf.freebase.com/ns/music.release_track.track_number> | "5" |
| 295 | <http://rdf.freebase.com/ns/g.1259vb5ld> | <http://rdf.freebase.com/ns/common.notable_for.display_name> | "Placering"@da |
| 296 | <http://rdf.freebase.com/ns/g.1q63pzhlz> | <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> | <http://rdf.freebase.com/ns/measurement_unit.dated_percentage> |
| 297 | <http://rdf.freebase.com/ns/music.single> | <http://rdf.freebase.com/ns/type.type.instance> | <http://rdf.freebase.com/ns/m.0xdrgt> |
| 298 | <http://rdf.freebase.com/ns/g.1256fv_n3> | <http://rdf.freebase.com/ns/common.notable_for.predicate> | "/type/object/type" |

In [4]:
cursor.execute("""
    CREATE TABLE fb
        (id SERIAL PRIMARY KEY,
        object varchar NOT NULL,
        relation varchar NOT NULL,
        subject varchar NOT NULL);""")

In [5]:
from tqdm import tqdm_notebook
import subprocess
import random

chunk_size = 10000

    
def insert_chunk(rows):
    insert_query = 'INSERT INTO fb (object, relation, subject) VALUES %s ON CONFLICT DO NOTHING;'
    psycopg2.extras.execute_values(
        cursor, insert_query, rows, template=None, page_size=100
    )
    
def is_relevant(split, all_mids):
    for token in split:
        if 'http://rdf.freebase.com/ns/m.' in token:
            mid = token.rstrip('>')
            mid = mid.lstrip('<')
            mid = mid.replace('http://rdf.freebase.com/ns/m.', '')
            if mid in all_mids:
                return True
    return False

rows = []
gzip = subprocess.Popen(['gzip', '-cdfq', fb], stdout=subprocess.PIPE)
for i, line in tqdm_notebook(enumerate(gzip.stdout), total=3130696870):
    line = line.decode('UTF-8')

    # Build Chunks
    split = line.split('\t')[:3]
    assert len(split) == 3, 'Malformed row'
    if is_relevant(split, all_mids):
        rows.append(tuple(split))
    
    # Insert Chunk
    if len(rows) > chunk_size:
        insert_chunk(rows)
        rows = []

insert_chunk(rows)

HBox(children=(IntProgress(value=0, max=3130696870), HTML(value='')))

Inserted 0.147 [10121012 of 68649962]
Inserted 0.145 [13981398 of 96195326]
Inserted 0.146 [16191619 of 110929518]
Inserted 0.149 [24012401 of 160714901]
Inserted 0.147 [24422442 of 166101943]
Inserted 0.145 [38903890 of 269002866]
Inserted 0.145 [65076507 of 449156111]
Inserted 0.144 [84428442 of 584497274]
Inserted 0.144 [96819681 of 672354882]
Inserted 0.145 [129812980 of 898132090]
Inserted 0.145 [164296428 of 1135407083]
Inserted 0.143 [177747773 of 1247022855]
Inserted 0.143 [195839582 of 1365145443]
Inserted 0.144 [204790477 of 1424759200]


KeyboardInterrupt: 

In [ ]:
connection.commit()

In [ ]:
cursor.execute("""
    CREATE INDEX fb_object_index ON fb(object);""")
connection.commit()

## Shut Down Connection

In [21]:
cursor.close()
connection.close()